In [40]:

from __future__ import print_function
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
import keras
from keras.utils import to_categorical
from sklearn.metrics import f1_score



BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
THIRD_MOD_DIR = "/home/abhinav/data/third_model/"
SECOND_MOD_DIR = "/home/abhinav/data/second_model/"
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 50
FOURTH_MOD_DIR = "/home/abhinav/data/fourth_model/"

In [ ]:
print('Loading data...')
[x_train, y_train] = pd.read_pickle(FOURTH_MOD_DIR + "train_data_10ratio") 
print(x_train.shape, 'train sequences')
print(y_train.shape, 'test sequences')

[x_test, y_test] = pd.read_pickle(FOURTH_MOD_DIR + "test_dataset") 

[x_val, y_val ] = pd.read_pickle(FOURTH_MOD_DIR + "validation_dataset")

In [20]:
[x_val_pos, y_val_pos] = pd.read_pickle(SECOND_MOD_DIR + "val_positive_data")
[x_val_neg, y_val_neg] = pd.read_pickle(SECOND_MOD_DIR + "val_negative_data")

In [ ]:
[x_train_pos, y_train_pos] = pd.read_pickle(FOURTH_MOD_DIR + "train_positive_data")

In [6]:
num_words = MAX_NB_WORDS

In [33]:
class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = to_categorical(self.model.predict_classes(self.validation_data[0],batch_size = 512))
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("Weighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

In [49]:
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


In [50]:
model.compile(optimizer=ndm, loss='binary_crossentropy', metrics=['acc'])

In [51]:
initial_weights = model.get_weights()

In [46]:
model.set_weights(initial_weights)

In [52]:
print('Train...')
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          class_weight = {0. : 1, 1. : 8},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train...
Train on 13150 samples, validate on 15165 samples
Epoch 1/4
15165/15165 [==============================] - 156s   
Weighted F1 score found on Validation dataset :  (array([ 0.99438639,  0.02788622]), array([ 0.7676    ,  0.60606061]), array([ 0.86639828,  0.05331911]), array([15000,   165]))
13150/13150 [==============================] - 739s - loss: 1.0568 - acc: 0.7494 - val_loss: 0.4697 - val_acc: 0.7658
Epoch 2/4
15165/15165 [==============================] - 157s   
Weighted F1 score found on Validation dataset :  (array([ 0.99127847,  0.05431755]), array([ 0.95473333,  0.23636364]), array([ 0.97266275,  0.08833522]), array([15000,   165]))
13150/13150 [==============================] - 737s - loss: 0.7428 - acc: 0.7949 - val_loss: 0.2366 - val_acc: 0.9469
Epoch 3/4
15165/15165 [==============================] - 156s   
Weighted F1 score found on Validation dataset :  (array([ 0.99466746,  0.05592303]), array([ 0.89533333,  0.56363636]), array([ 0.94239001,  0.10175055]),

In [19]:
model.evaluate(x_test[:-175],y_test[:-175],batch_size=512)

15000/15000 [==============================] - 167s   


[0.38012623756726582, 0.82513333349227902]

In [21]:
model.evaluate(x_val_pos,y_val_pos,batch_size=512)

350/350 [==============================] - 4s


[0.84063678979873657, 0.51714283227920532]

In [22]:
model.evaluate(x_val_neg,y_val_neg,batch_size=512)

30000/30000 [==============================] - 337s   


[0.37901017692883809, 0.82436666666666669]

In [24]:
model.evaluate(x_train_pos,y_train_pos,batch_size=512)

1228/1228 [==============================] - 13s    


[0.20455269632005535, 0.96416937664199731]

In [ ]:
+